# Building an Agentic RAG Router with Azure OpenAI and Azure AI Search

This notebook demonstrates how to build a basic agentic RAG (Retrieval-Augmented Generation) system using Azure OpenAI, Azure AI Search, and Bing Search. The agent can intelligently switch between searching a private knowledge base (using Azure AI Search) and public web content (using Bing Search) based on the query context.

## What is an Agentic RAG Router?
An Agentic RAG Router is a system where an intelligent agent dynamically routes queries to appropriate tools or data sources, enhancing the capabilities of Large Language Models (LLMs). The router determines which retrieval mechanism is best suited to answer each query.


## Prerequisites
- An Azure OpenAI service instance
- An Azure AI Search service with an existing search index
- A Bing Search API key
- Python environment with required packages (requirements.txt provided)

## Architecture Overview
This implementation follows a hybrid search pattern where:
1. Private data is indexed in Azure AI Search (in this example, a FIFA Legal Handbook)
2. Public data is accessed through Bing Search API
3. An Azure OpenAI agent orchestrates between these sources based on query context

The agent uses function calling to determine which search tool to use and how to combine information from multiple sources when needed.

## Environment Setup
First, we'll install the required packages from requirements.txt. This includes the Azure SDK packages and other dependencies.

```bash
pip install openai azure-search-documents python-dotenv rich requests
```

Or just run the next line

In [ ]:
!pip install -r requirements.txt

## Step 1: Setup & Imports

In this step, we:
1. Import required libraries for Azure services, OpenAI, and utilities
2. Set up configuration variables (preferably via environment variables)
3. Initialize the Azure OpenAI client

Key configurations include:
- Azure OpenAI settings (endpoint, API key, model name)
- Azure AI Search settings (endpoint, key, index name)
- Bing Search API settings

Best practice: Store these configurations in environment variables rather than hardcoding them in the notebook.

In [ ]:
import os
import json
import requests
from rich.console import Console
from rich.panel import Panel

# Azure OpenAI configuration
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY", "your-azure-openai-api-key")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-10-21")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT", "https://your-azure-openai-endpoint.openai.azure.com/")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME", "gpt-4o")  # update as needed

# Azure AI Search configuration
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT", "https://your-search-service.search.windows.net")
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY", "your-azure-search-key")
SEARCH_INDEX_NAME = "fifa-legal-handbook"

# Bing Search configuration
BING_SEARCH_API_KEY = os.getenv("BING_SEARCH_API_KEY", "your-bing-search-api-key")
BING_SEARCH_ENDPOINT = "https://api.bing.microsoft.com/v7.0/search"

# Import Azure libraries for search
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizableTextQuery

# Import Azure OpenAI client
from openai import AzureOpenAI

console = Console()

In [12]:
# Initialize the Azure OpenAI client
openai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
)

## Step 2: Define Search Functions

Here we implement two core search functions that our agent will use:

1. **search_azure_ai_search(query: str)**: 
   - Searches the private Azure AI Search index ("fifa-legal-handbook")
   - Uses hybrid search combining vector and semantic search capabilities
   - Returns relevant chunks from the private knowledge base

2. **search_bing(query: str)**:
   - Searches the public web using Bing Search API
   - Returns relevant snippets from web results
   - Useful for current events and public information

Each function includes logging to track which tool is being used and the specific query being processed.

In [ ]:
def search_azure_ai_search(query: str) -> str:
    """
    Searches the Azure AI Search index 'fifa-legal-handbook' using a hybrid semantic approach.
    This function retrieves legal rules, regulations, and disciplinary information from the FIFA Legal Handbook.
    Intended for assisting FIFA referees with queries about legal guidelines.
    """
    credential = AzureKeyCredential(AZURE_SEARCH_KEY)
    client = SearchClient(
        endpoint=AZURE_SEARCH_ENDPOINT,
        index_name=SEARCH_INDEX_NAME,
        credential=credential,
    )

    results = client.search(
        search_text=query,
        vector_queries=[
            VectorizableTextQuery(
                text=query, k_nearest_neighbors=50, fields="text_vector"
            )
        ],  # Update with your fields
        query_type="semantic",
        semantic_configuration_name="default",  # Update with your semantic configuration
        search_fields=["chunk"],  # Update with your content field name
        top=50,
        include_total_count=True,
    )

    retrieved_texts = []
    for result in results:
        content = result.get("chunk", "")
        retrieved_texts.append(content)

    context_str = (
        "\n".join(retrieved_texts) if retrieved_texts else "No documents found."
    )
    console.print(
        Panel(f"Tool Invoked: Azure AI Search\nQuery: {query}", style="bold yellow")
    )
    return context_str


def search_bing(query: str) -> str:
    """
    Searches Bing using the Bing Search API.
    Returns a concatenated string of result snippets.
    """
    headers = {"Ocp-Apim-Subscription-Key": BING_SEARCH_API_KEY}
    params = {"q": query, "textDecorations": True, "textFormat": "Raw"}
    response = requests.get(BING_SEARCH_ENDPOINT, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        if "webPages" in data and "value" in data["webPages"]:
            snippets = [item.get("snippet", "") for item in data["webPages"]["value"]]
            result_text = "\n".join(snippets)
        else:
            result_text = "No Bing results found."
    else:
        result_text = f"Bing search failed with status code {response.status_code}."

    console.print(
        Panel(f"Tool Invoked: Bing Search\nQuery: {query}", style="bold magenta")
    )
    return result_text

In [34]:
# Test Azure AI Search (FIFA Legal Handbook)
test_query_azure = "What are the FIFA disciplinary rules for match-fixing?"
azure_search_result = search_azure_ai_search(query=test_query_azure)
console.print(
    Panel(
        azure_search_result, 
        title="Azure AI Search Test Output (FIFA Legal Handbook)", 
        style="bold cyan",
        border_style="white"
    )
)

# Test Bing Search (Public Web)
test_query_bing = "Recent incidents of match-fixing in international football"
bing_search_result = search_bing(query=test_query_bing)
console.print(
    Panel(
        bing_search_result, 
        title="Bing Search Test Output (Public News)", 
        style="bold cyan",
        border_style="white"
    )
)


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Tool Invoked: Azure AI Search                                                                                   │
│ Query: What are the FIFA disciplinary rules for match-fixing?                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────── Azure AI Search Test Output (FIFA Legal Handbook) ───────────────────────────────╮
│ with, among others, the principles of sporting integrity                                                        │
│ and fair play.                                                                                                  │
│                                                                                                                 │
│ This work encompasses the fight against match-fixing, the use of doping and any other forms                     │
│ of cheating in sport, alongside the strengthening of ethics with improvements in transparency,                  │
│ good governance and accountability.                                                                             │
│                                                                                                                 │
│ The  is committed to promoting and upholding the integrity of                                                   │
│ our sport across the whole football family. We stand for integrity in football, for a sport you can             │
│ trust. Having clean competitions is our greatest asset. Matches should be decided on the football               │
│ pitch, not on the gambling market.                                                                              │
│                                                                                                                 │
│ Match-fixing is an intentional arrangement, act or omission aimed at an improper alteration of the              │
│ result or the course of a sports competition in order to remove all or part of the unpredictable                │
│ nature of the sports competition with a view to obtaining an undue benefit. This is against the                 │
│ rules of the game and often the law. We want to preserve what is great in the beautiful game                    │
│ and have devised rules regarding betting in football to help protect the integrity and future of                │
│ football. These rules apply to everyone involved in football.                                                   │
│                                                                                                                 │
│ Our disciplinary bodies are responsible for ensuring that these fundamental principles of football are          │
│ respected and help to promote integrity in sport. Therefore, we will always adopt a zero-tolerance              │
│ policy to match-fixing. We have opted for a combined repressive and preventive approach                         │
│ comprising a wide range of actions and measures. If you attempt match-fixing, breach the  betting rules, or     │
│ pass on inside information, you are likely to get caught                                                        │
│ and be charged.                                                                                                 │
│                                                                                                                 │
│ To support this, we have a set of rules to which all club officials, players, match officials and  staff must   │
│ adhere. These rules apply to betting, the use of inside information                                             │
│ and match manipulation. The rules help ensure that any individual involved in a football match                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Tool Invoked: Bing Search                                                                                       │
│ Query: Recent incidents of match-fixing in international football                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────── Bing Search Test Output (Public News) ─────────────────────────────────────╮
│ In 1964, the great British football betting scandal of the 1960s was uncovered: a betting ring organized by   │
│ Jimmy Gauld and involving several Football League players had been fixing matches. The most famous        │
│ incident involved three Sheffield Wednesday players, including two England international players, who were    │
│ subsequently banned from football for life and imprisoned after it was discovered ...                         │
│ Match-fixing has a long history of over 100 years. Some of the biggest match-fixing scandals took place in  │
│ football, basketball, cricket, and baseball. Pakistani cricket fans in Lahore on 4 November 2011 following    │
│ the arrest of their players over match-fixing allegations in Britain. Photo: Arif Ali Source: Getty Images    │
│ This article analyses the approach of sporting governing bodies, courts and tribunals to recent incidents   │
│ of alleged corruption in sport. These include the sanctions imposed by FIFA on a Ukrainian football referee   │
│ and a Macedonian football club President, who were both handed lifetime bans as a result of their involvement │
│ with unlawful betting syndicates, decisions which were ultimately upheld ...                                    │
│ The Nigerian Football Association handed down lifetime bans to all four teams and the match officials who   │
│ partook in this instance of match-fixing. 8. Calciopoli (2006)                                                │
│ A number of clubs across the top two tiers of Italian football were implicated in the Totonero match-fixing │
│ scandal, with Serie A giants Milan and Lazio suffering the most severe sanction ...                             │
│ Match fixing brings disrepute to any sport and any team that is involved in it. The actions of a few are      │
│ suffered by many, with football having more than its fair share of controversies...                           │
│ Macarthur FC player Ulises Davila passes the ball during an A-League match against Melbourne City in Sydney,  │
│ March 1, 2024. Davila and fellow players Kearyn Baccus and Clayton Lewis have been arrested, Friday, May 17,    │
│ 2024, over their alleged involvement in a the betting fix, which police say has led to hundreds of thousands of │
│ dollars being paid out in winnings.                                                                             │
│ The International Betting Integrity Association (IBIA) says football and tennis accounted for over half of  │
│ suspicious sporting incidents in the third quarter of 2023. Advertisement                                     │
│ The issue of match fixing in association football has been described, in 2013, by Chris Eaton, the former   │
│ Head of Security of FIFA (the sport's world governing body), as a "crisis", [1] while UEFA's president Michel   │
│ Platini has said that if it continues, "football is dead". [2] Zhang Jilong, president of the Asian           │
│ Football Confederation, has stated that it is a "pandemic". [3]                                               │
│ Here we take a look at the top 5 most famous incidents of match-fixing in world football. Corruption in   │
│ football is a disease that has unfortunately reared its ugly head more than once in recent years. Here we   │
│ take a look at the top 5 most famous incidents of match-fixing in world football.                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Step 3: Define Function Descriptions for OpenAI

Here we define the function schemas that Azure OpenAI will use to decide which search tool to call. These descriptions help the model understand:
- When to use each function
- What parameters they accept
- What kind of information they return

This metadata helps the agent make intelligent decisions about which tool to use based on the query context.

In [36]:
functions = [
    {
        "name": "search_azure_ai_search",
        "description": (
            "Use this function to search the private FIFA Legal Handbook for legal guidelines, "
            "regulations, and disciplinary rules. This is used for answering questions related to FIFA rules "
            "and legal matters to assist referees in making informed decisions."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query to retrieve relevant legal data from the FIFA Legal Handbook",
                },
            },
            "required": ["query"],
        },
    },
    {
        "name": "search_bing",
        "description": (
            "Use this function to perform a real-time web search for public information, news, and current events "
            "related to football. This helps provide context for queries about match incidents or disciplinary events."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query to retrieve recent news and public data from the web",
                },
            },
            "required": ["query"],
        },
    },
]


## Step 4: Process User Queries with Intelligent Routing

This section implements the core agent logic that:
1. Receives a user query
2. Determines which tool(s) to use based on the query context
3. Calls the appropriate function(s)
4. Synthesizes a final response

The agent can:
- Use Azure AI Search for questions about FIFA rules and regulations
- Use Bing Search for current events and public information
- Combine information from both sources when needed

In [37]:
# This system prompt guides the agent to leverage the appropriate tool based on the query.

system_prompt = (
    "You are an expert assistant for FIFA referees. You have access to private legal data "
    "from the FIFA Legal Handbook and real-time public information via Bing Search. "
    "When a query concerns specific FIFA rules, legal guidelines, or disciplinary matters, "
    "prioritize retrieving information from the FIFA Legal Handbook using Azure AI Search. "
    "For queries about current events, match incidents, or general news, use Bing Search. "
    "If both aspects are relevant, synthesize answers from both sources to provide a comprehensive response."
)

In [ ]:
def run_basic_agent(user_query: str):
    """
    Executes the referee query by including the system prompt, sending the message to the OpenAI client,
    routing to the appropriate tool (Azure AI Search for legal data or Bing Search for public news), and printing the final answer.
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ]
    
    # Call the chat completions API with the function definitions.
    response = openai_client.chat.completions.create(
        model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,  
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    
    response_message = response.choices[0].message
    
    if response_message.function_call is not None:
        function_name = response_message.function_call.name
        function_args = json.loads(response_message.function_call.arguments)
        query_for_function = function_args.get("query")
    
        if function_name == "search_azure_ai_search":
            function_response = search_azure_ai_search(query=query_for_function)
        elif function_name == "search_bing":
            function_response = search_bing(query=query_for_function)
        else:
            function_response = "Function not found"
    
        # Send the function response back to the model for final answer synthesis.
        second_response = openai_client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
            messages=[
                *messages,  # Original system and user messages
                response_message.model_dump(), 
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
    
        final_answer = second_response.choices[0].message.content
        console.print(
            Panel(
                f"Function Called: {function_name}\n\nFinal Answer: {final_answer}",
                title="Function Call & Final Response",
                style="bold green",
                border_style="yellow",
            )
        )
    else:
        final_answer = response_message.content
        console.print(
            Panel(
                final_answer,
                title="Direct Response",
                style="bold green",
                border_style="yellow",
            )
        )

## Testing the Agent

Below are example queries that demonstrate the agent's ability to:
1. Route questions about FIFA regulations to Azure AI Search
2. Route questions about recent events to Bing Search
3. Synthesize information from both sources when appropriate

Try different queries to test the agent's routing capabilities!

In [ ]:
# Example usage: I expect this query ot trigger the Azure AI Search function, let's see! 
run_basic_agent("What are the titles of the FIFA regulations that are contained in the FIFA legal hand book, and what are the edition years of those regulations?")

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Tool Invoked: Azure AI Search                                                                                   │
│ Query: FIFA regulations titles and edition years                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── Function Call & Final Response ─────────────────────────────────────────╮
│ Function Called: search_azure_ai_search                                                                         │
│                                                                                                                 │
│ Final Answer: The FIFA Legal Handbook contains several regulations, each with specific edition years. Here are  │
│ some of the notable titles and their respective edition years:                                                  │
│                                                                                                                 │
│ 1. **FIFA Anti-Doping Regulations** - 2019 Edition                                                              │
│ 2. **FIFA Match Agents Regulations** - 2003 Edition                                                             │
│ 3. **FIFA Disciplinary Code** - (Other editions not specified, but typically updated around every few years)    │
│ 4. **Regulations Governing the Transfer of Players** - (Edition varies, often updated)                          │
│ 5. **FIFA Club Licensing Regulations** - (Edition varies, often updated)                                        │
│                                                                                                                 │
│ The specific regulations can have multiple amendments and updates over different editions, and the exact        │
│ edition year may vary for the regulations not explicitly listed. If you need more detailed information or       │
│ specific articles from these regulations, please let me know!                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# I expect the agent to use Bing Search for this query, let's see! 
run_basic_agent("What are the most recent controversies involving FIFA's Football Agent Regulations, from the last 3 months?")

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Tool Invoked: Bing Search                                                                                       │
│ Query: recent controversies FIFA Football Agent Regulations last 3 months                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── Function Call & Final Response ─────────────────────────────────────────╮
│ Function Called: search_bing                                                                                    │
│                                                                                                                 │
│ Final Answer: In the last three months, notable controversies have emerged regarding FIFA's Football Agent      │
│ Regulations (FFAR), which officially came into effect on October 1, 2023. Here are the key points of these      │
│ controversies:                                                                                                  │
│                                                                                                                 │
│ 1. **Court of Arbitration for Sport (CAS) Ruling**: In late July 2023, CAS ruled in favor of FIFA amid appeals  │
│ from football agents challenging the legality of the new FFAR. This ruling reinforced FIFA's authority over the │
│ new regulations but did not quell the dissent among agents.                                                     │
│                                                                                                                 │
│ 2. **Legal Challenges and Suspension**: Following a series of legal challenges by agents and agents’            │
│ associations in various jurisdictions, FIFA announced the partial suspension of these regulations on December   │
│ 30, 2023. The suspension was implemented pending a decision from the European Court of Justice after multiple   │
│ injunctions were filed against the enforcement of certain rules.                                                │
│                                                                                                                 │
│ 3. **Key Disputed Elements**: The FFAR introduced controversial provisions, including caps on commissions and   │
│ limits on agents representing multiple players in the same transaction. These aspects have drawn significant    │
│ criticism from agents, leading to widespread legal opposition.                                                  │
│                                                                                                                 │
│ 4. **Implementation Uncertainty**: Despite the regulations theoretically aiming to standardize the profession   │
│ of football agents globally, their practical implementation has faced halts and skepticism, with national       │
│ federations, such as the English FA, grappling with compliance and adaptation to the changing rules.            │
│                                                                                                                 │
│ The landscape remains uncertain as FIFA must navigate ongoing legal disputes while seeking to re-establish      │
│ regulations that address both regulatory objectives and the interests of football agents.                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯